# Grid Movement Example

**Spatial navigation with Miniverse's Tier 2 environment.**

This notebook shows how agents can move through a 2D grid world with:
- **Tile-based movement** - agents navigate x,y coordinates
- **Collision detection** - walls and obstacles block movement
- **Pathfinding** - A* algorithm finds routes around obstacles
- **Spatial awareness** - agents perceive their location and nearby tiles

We'll create a simple warehouse where a robot navigates from one corner to another.

## Setup

In [1]:
import os
from datetime import datetime, timezone
from miniverse import (
    Orchestrator, AgentProfile, AgentStatus, WorldState,
    ResourceState, EnvironmentState, SimulationRules, Stat
)
from miniverse.cognition import AgentCognition, LLMExecutor
from miniverse.environment import EnvironmentGrid, GridTile, grid_shortest_path

print('✅ Imports complete')

✅ Imports complete


## Create the Grid World

We'll create a 10x10 warehouse with:
- **Walls** around the perimeter
- **Storage racks** in the middle (obstacles)
- **Robot** starting at bottom-left
- **Goal** at top-right

In [2]:
# Create 10x10 grid
width, height = 10, 10
grid = EnvironmentGrid(width=width, height=height)

# Add walls around perimeter
for x in range(width):
    grid.set_tile(x, 0, GridTile(tile_type='wall', passable=False))  # Bottom wall
    grid.set_tile(x, height-1, GridTile(tile_type='wall', passable=False))  # Top wall

for y in range(height):
    grid.set_tile(0, y, GridTile(tile_type='wall', passable=False))  # Left wall
    grid.set_tile(width-1, y, GridTile(tile_type='wall', passable=False))  # Right wall

# Add storage racks (obstacles) in the middle
obstacles = [
    (3, 3), (3, 4), (3, 5),  # Vertical rack
    (6, 3), (6, 4), (6, 5),  # Another vertical rack
    (4, 6), (5, 6),          # Horizontal rack
]

for x, y in obstacles:
    grid.set_tile(x, y, GridTile(tile_type='storage_rack', passable=False))

# Mark goal location
grid.set_tile(8, 8, GridTile(tile_type='goal', passable=True, properties={'goal': True}))

print('✅ Grid world created (10x10)')
print(f'   Obstacles: {len(obstacles)} storage racks')
print(f'   Start: (1, 1)')
print(f'   Goal: (8, 8)')

# Visualize the grid
print('\n=== Warehouse Layout ===')
print('Legend: # = wall, X = rack, G = goal, . = floor\n')

for y in range(height-1, -1, -1):  # Top to bottom
    row = ''
    for x in range(width):
        tile = grid.get_tile(x, y)
        if tile.tile_type == 'wall':
            row += '# '
        elif tile.tile_type == 'storage_rack':
            row += 'X '
        elif tile.tile_type == 'goal':
            row += 'G '
        else:
            row += '. '
    print(f'{y:2d} {row}')

print('   ' + '  '.join(str(x) for x in range(width)))

AttributeError: 'EnvironmentGrid' object has no attribute 'set_tile'

## Pathfinding Demo

Let's use A* pathfinding to calculate the optimal route from start to goal.

In [ ]:
start = (1, 1)
goal = (8, 8)

# Calculate path using A* algorithm
path = grid_shortest_path(grid, start, goal)

if path:
    print(f'✅ Path found! {len(path)} steps')
    print(f'\nRoute: {start} → {goal}')
    print(f'Steps: {" → ".join(str(p) for p in path[:5])}...')
    
    # Visualize path on grid
    print('\n=== Path Visualization ===')
    print('Legend: # = wall, X = rack, * = path, S = start, G = goal\n')
    
    path_set = set(path[1:-1])  # Exclude start and goal for display
    
    for y in range(height-1, -1, -1):
        row = ''
        for x in range(width):
            coord = (x, y)
            tile = grid.get_tile(x, y)
            
            if coord == start:
                row += 'S '
            elif coord == goal:
                row += 'G '
            elif coord in path_set:
                row += '* '
            elif tile.tile_type == 'wall':
                row += '# '
            elif tile.tile_type == 'storage_rack':
                row += 'X '
            else:
                row += '. '
        print(f'{y:2d} {row}')
    
    print('   ' + '  '.join(str(x) for x in range(width)))
else:
    print('❌ No path found!')

## Simulation with Movement

Now let's create a simple simulation where the robot follows the path.

We'll use deterministic movement (no LLM needed) - the robot just follows the calculated path.

In [ ]:
class WarehouseRules(SimulationRules):
    """Robot follows pre-calculated path through warehouse."""
    
    def __init__(self, path):
        self.path = path
        self.path_index = 0
    
    def apply_tick(self, state, tick):
        updated = state.model_copy(deep=True)
        
        # Move robot along path
        if self.path_index < len(self.path):
            current_pos = self.path[self.path_index]
            
            # Update robot's location
            robot = next(a for a in updated.agents if a.agent_id == 'robot')
            robot.location = f'{current_pos[0]},{current_pos[1]}'
            
            # Update progress metric
            progress = updated.resources.get_metric('progress', default=0, unit='%')
            progress.value = (self.path_index / len(self.path)) * 100
            
            self.path_index += 1
        
        updated.tick = tick
        return updated
    
    def validate_action(self, action, state):
        return True

# Create world state with robot at start position
world_state = WorldState(
    tick=0,
    timestamp=datetime.now(timezone.utc),
    environment=EnvironmentState(
        metrics={},
        grid={'width': width, 'height': height, 'tiles': 'warehouse_grid'}
    ),
    resources=ResourceState(metrics={
        'progress': Stat(value=0, unit='%', label='Delivery Progress'),
        'distance_remaining': Stat(value=len(path), unit='tiles', label='Distance to Goal')
    }),
    agents=[
        AgentStatus(
            agent_id='robot',
            display_name='Warehouse Robot',
            location=f'{start[0]},{start[1]}'
        )
    ]
)

# Create agent profile
robot_profile = AgentProfile(
    agent_id='robot',
    name='Warehouse Robot',
    age=0,
    background='Autonomous delivery robot',
    role='warehouse_robot',
    personality='efficient, precise, goal-oriented',
    skills={'navigation': 'expert', 'pathfinding': 'expert'},
    goals=['Deliver package to goal location', 'Avoid obstacles', 'Minimize travel time'],
    relationships={}
)

# Simple deterministic cognition (no LLM needed for this example)
from miniverse.cognition import SimpleExecutor

cognition = {
    'robot': AgentCognition(
        executor=SimpleExecutor(default_action='move')
    )
}

agent_prompts = {
    'robot': 'Follow the calculated path to the goal.'
}

print('✅ Simulation configured')
print(f'   Path length: {len(path)} steps')
print(f'   Starting position: {start}')
print(f'   Goal position: {goal}')

## Run the Simulation

Watch the robot navigate through the warehouse!

In [ ]:
# Create orchestrator
orchestrator = Orchestrator(
    world_state=world_state,
    agents={'robot': robot_profile},
    world_prompt='',
    agent_prompts=agent_prompts,
    simulation_rules=WarehouseRules(path),
    agent_cognition=cognition,
    llm_provider=None,  # No LLM needed
    llm_model=None
)

# Run simulation (one tick per step in path)
num_ticks = min(10, len(path))  # Show first 10 steps
print(f'🤖 Running robot simulation ({num_ticks} ticks)...\n')

result = await orchestrator.run(num_ticks=num_ticks)

print('\n✅ Simulation complete!')

# Show final state
final = result['final_state']
robot = final.agents[0]
progress = final.resources.metrics['progress']

print(f'\nFinal position: {robot.location}')
print(f'Progress: {progress.value:.1f}%')
print(f'Steps taken: {final.tick}')

if final.tick >= len(path):
    print('\n🎯 Goal reached!')
else:
    print(f'\n⏸️  Simulation paused ({len(path) - final.tick} steps remaining)')

## Visualize Robot's Journey

Let's trace the robot's actual path through the simulation.

In [ ]:
# Get robot's movement history from persistence
run_id = result['run_id']
visited_positions = []

for tick in range(1, final.tick + 1):
    state = await orchestrator.persistence.get_state(run_id, tick)
    if state:
        robot = state.agents[0]
        x, y = map(int, robot.location.split(','))
        visited_positions.append((x, y))

print(f'=== Robot Journey ({len(visited_positions)} positions) ===')
print(f'Route: {" → ".join(str(p) for p in visited_positions[:5])}...')

# Visualize journey
print('\n=== Journey Visualization ===')
print('Legend: # = wall, X = rack, • = visited, R = current, S = start, G = goal\n')

visited_set = set(visited_positions[:-1])  # All except current
current_pos = visited_positions[-1] if visited_positions else start

for y in range(height-1, -1, -1):
    row = ''
    for x in range(width):
        coord = (x, y)
        tile = grid.get_tile(x, y)
        
        if coord == current_pos:
            row += 'R '
        elif coord == start:
            row += 'S '
        elif coord == goal:
            row += 'G '
        elif coord in visited_set:
            row += '• '
        elif tile.tile_type == 'wall':
            row += '# '
        elif tile.tile_type == 'storage_rack':
            row += 'X '
        else:
            row += '. '
    print(f'{y:2d} {row}')

print('   ' + '  '.join(str(x) for x in range(width)))

## 💡 What You Just Saw

**Grid-based spatial navigation in Miniverse:**

### Key Features Demonstrated:
1. **Tier 2 Environment** - 2D grid with x,y coordinates
2. **Tile System** - Different tile types (floor, wall, obstacle)
3. **Collision Detection** - Walls and racks block movement
4. **A* Pathfinding** - `grid_shortest_path()` finds optimal routes
5. **Location Tracking** - Agents have `location` attribute (x,y)
6. **State Persistence** - Can replay and visualize agent movement

### The Grid API:
```python
# Create grid
grid = EnvironmentGrid(width=10, height=10)

# Set tiles
grid.set_tile(x, y, GridTile(tile_type='wall', passable=False))

# Pathfinding
path = grid_shortest_path(grid, start=(1,1), goal=(8,8))

# Agent location
agent.location = '5,7'  # x,y as string
```

### Use Cases:
- **Robotics simulations** - warehouse, delivery, exploration
- **Game AI** - NPCs navigating game worlds
- **Social simulations** - agents moving through buildings (Stanford style)
- **Evacuation modeling** - crowd movement through spaces

### Next Steps:
- Add LLM-based decision making for dynamic obstacle avoidance
- Multiple agents coordinating in shared space
- Dynamic environments (doors, changing obstacles)
- Combine with Tier 1 (logical graph) for multi-level buildings

Check out `docs/architecture/environment.md` for the full environment API!